In [48]:
import pandas as pd
class JsonURLLoader:
    """
    Load JSON data from a URL.
    """

    def __init__(self, url):
        self.url = url
        
    def load(self) -> dict:
        """
        Load JSON data from a URL.

        Returns:
            dict: JSON data.
        """
        import json
        import urllib.request
        with urllib.request.urlopen(self.url) as url:
            data = json.loads(url.read().decode())
        return data

class JsonFileLoader:
    """ 
    Load JSON data from a file.
    """
    def __init__(self, filename):
        self.filename = filename

    def load(self) -> dict:
        """
        Load JSON data from a file.
        Returns:
            dict : JSON data.
        """
        import json
        with open(self.filename) as json_file:
            data = json.load(json_file)
        return data

class JsonLoaderFactory:
    """
    Factory for creating JSON loaders.
    """
    def get_loader(self, source) -> JsonURLLoader or JsonFileLoader:
        """
        Create a JSON loader.

        Args:
            source (str): URL or filename.

        Raises:
            ValueError: If the source is not a URL or filename.

        Returns:
            _type_: JsonURLLoader or JsonFileLoader
        """
        
        if source.startswith(("http://", "https://")) and source.endswith(".json"):
            return JsonURLLoader(source)
        elif source.endswith(".json"):
            return JsonFileLoader(source)
        else:
            raise ValueError(source)

In [49]:
class JsonToDfConverter:
    """
    Convert JSON data to a Pandas DataFrame.
    """
    def __init__(self, json):
        self.json = json

    def convert(self) -> pd.DataFrame:
        """
        Convert JSON data to a Pandas DataFrame.

        Returns:
            pd.DataFrame : Pandas DataFrame
        """
        
        import pandas as pd
        df = pd.json_normalize(self.json)
        return df

class DFJoiner:
    """
    Join two Pandas DataFrames.
    """
    def __init__(self, df1, df2, on):
        self.df1 = df1
        self.df2 = df2
        self.on = on

    def join(self) -> pd.DataFrame:
        """
        Join two Pandas DataFrames.

        Returns:
            pd.DataFrame : Pandas DataFrame
        """
        joined_df = self.df1.join(self.df2.set_index('id'), on=self.on)
        return joined_df

In [50]:
from dataclasses import dataclass
from abc import ABC, abstractmethod
import pandas as pd

@dataclass
class QueryData:
    """
    QueryData is a dataclass that contains the data needed to calculate the commission
    """
    sales_rep_name: str
    start_date: str
    end_date: str
    
class CommissionStrategy(ABC):
    """
    CommissionStrategy is an abstract class that defines the CalculateCommission method
    """

    @abstractmethod
    def CalculateCommission(self,data:QueryData) -> float:
        """
        CalculateCommission is an abstract method that calculates the commission

        Args:
            data (QueryData): QueryData is a dataclass that contains the data needed to calculate the commission

        Returns:
            float: The total commission
        """
        pass
    
class FixedCommissionStrategy(CommissionStrategy):
    """
    FixedCommissionStrategy is a class that calculates the commission for a fixed commission type
    """

    def CalculateCommission(self,data:QueryData) -> float:
        """
        CalculateCommission is a method that calculates the commission for a fixed commission type

        Args:
            data (QueryData): QueryData is a dataclass that contains the data needed to calculate the commission

        Returns:
            float: The total commission
        """        
        #Get the sales rep name from the data
        sales_rep_name = data.sales_rep_name

        #Get the start date from the data
        start_date = pd.to_datetime(data.start_date)

        #Get the end date from the data
        end_date = pd.to_datetime(data.end_date)

        #read the sales data from the sales_data.csv file
        sales_data = pd.read_csv("sales_data.csv")

        #change the date column to a datetime object
        sales_data['date'] = pd.to_datetime(sales_data['date'])

        #filter the filtered_df to only include deals that were closed by the sales_rep_name
        filtered_df = sales_data[sales_data['sales_rep_name'] == sales_rep_name]
        
        #filter the merged_df to only include deals that were closed between the start and end date
        final_df = filtered_df[(filtered_df['date'] >= start_date) & (filtered_df['date'] <= end_date)]
    
        #calculate the total commission for the sales_rep_name
        commission = final_df['quantity_products_sold'] * final_df['product_amount'] * final_df['commission_rate']
        total_commission = commission.sum()
        
        return '{:.2f}'.format(total_commission)

class CommissionStrategyFactory:
    """
    CommissionStrategyFactory is a class that creates a commission strategy
    """
    def GetCommissionStrategy(self,commissionType: str="Fixed") -> CommissionStrategy:
        """
        GetCommissionStrategy is a method that creates a commission strategy

        Args:
            commissionType (str, optional): Defaults to "Fixed".

        Raises:
            Exception: Invalid commission type

        Returns:
            CommissionStrategy: A commission strategy
        """
        if commissionType == "Fixed":
            return FixedCommissionStrategy()
        else:
            raise Exception("Invalid commission type")

class CalcuateCommission:
    """
    CalcuateCommission is a class that calculates the commission
    """
    def __init__(self, commissionStrategyFactory: CommissionStrategyFactory):
        self._commissionStrategyFactory = commissionStrategyFactory

    def CalculateCommission(self, data:QueryData,commissionType: str) -> float:
        """
        CalculateCommission is a method that calculates the commission

        Args:
            data (QueryData): QueryData is a dataclass that contains the data needed to calculate the commission
            commissionType (str): The commission type

        Returns:
            float: The total commission
        """
        strategy = self._commissionStrategyFactory.GetCommissionStrategy(commissionType)
        return strategy.CalculateCommission(data)

In [51]:
import datetime
def load_json(source) -> dict:
    """
    Load JSON data from a file or URL.

    Args:
        source (str): Path to a file or URL.

    Returns:
        dict : JSON data.
    """
    factory = JsonLoaderFactory()
    loader = factory.get_loader(source)
    return loader.load()

def validate_data(data) -> bool:
    """
    Validate the data

    Args:
        data (QueryData): QueryData is a dataclass that contains the data needed to calculate the commission

    Raises:
        ValueError: data cannot be null
        ValueError: data cannot be empty
        TypeError: sales_rep_name must be a string
        TypeError: start_date must be a string
        TypeError: end_date must be a string
        ValueError: start_date must be in the format YYYY-MM-DD
        ValueError: end_date must be in the format YYYY-MM-DD
        ValueError: start_date must be before end_date
        ValueError: sales_rep_name must be in the sales_data.csv file

    Returns:
        bool: True if the data is valid
    """

    #validate the data is not null
    if data is None:
        raise ValueError("data cannot be null")
    
    #validate the data is not empty
    if data.sales_rep_name == "" or data.start_date == "" or data.end_date == "":
        raise ValueError("data cannot be empty")

     #validate the datatypes
    if not isinstance(data.sales_rep_name, str):
        raise TypeError("sales_rep_name must be a string")
    if not isinstance(data.start_date, str):
        raise TypeError("start_date must be a string")
    if not isinstance(data.end_date, str):
        raise TypeError("end_date must be a string")

    #validate the date format
    try:
        pd.to_datetime(data.start_date)
    except ValueError:
        raise ValueError("start_date must be in the format YYYY-MM-DD")
    try:
        pd.to_datetime(data.end_date)
    except ValueError:
        raise ValueError("end_date must be in the format YYYY-MM-DD")
    
    #validate the start date is before the end date
    if pd.to_datetime(data.start_date) > pd.to_datetime(data.end_date):
        raise ValueError("start_date must be before end_date")

    ''' 
    #validate start_date and end_date are greater than current date
    # the below validations is not needed and does not work since the data is mocked and the dates are made up
    # if pd.to_datetime(data.start_date) > pd.to_datetime(datetime.now().strftime("%Y-%m-%d")):
    #     raise ValueError("start_date must be before current date")
    
    # if pd.to_datetime(data.end_date) > pd.to_datetime(datetime.now().strftime("%Y-%m-%d")):
    #     raise ValueError("end_date must be before current date")
        
    '''

    #validate the sales rep name is in the sales_data.csv file
    sales_data = pd.read_csv("sales_data.csv")
    sales_rep_names = sales_data['sales_rep_name'].unique()
    if data.sales_rep_name not in sales_rep_names:
        raise ValueError("sales_rep_name must be in the sales_data.csv file")

    return True

def calculate_commission(sales_rep_name, start_date, end_date) -> float:
    """
    Calculate commission for a sales rep.

    Args:
        sales_rep_name (str): Sales rep name.
        start_date (str): Start date.
        end_date (str): End date.

    Returns:
        float: Commission.
    """
    #input the data into a QueryData dataclass
    data = QueryData(sales_rep_name, start_date, end_date)

    #validate the data
    validate_data(data)
   
    #calculate the commission
    commissionType = "Fixed"
    commissionStrategyFactory = CommissionStrategyFactory()
    calcuateCommission = CalcuateCommission(commissionStrategyFactory)
    commission = calcuateCommission.CalculateCommission(data,commissionType)
    return commission

def PrintCommission(sales_rep_name, start_date, end_date):
    """
    Print the commission for a sales rep.

    Args:
        sales_rep_name (str): Sales rep name.
        start_date (str): Start date.
        end_date (str): End date.
    """
    commission = calculate_commission(sales_rep_name, start_date, end_date)
    print(f"Commission for {sales_rep_name} between {start_date} and {end_date} is {commission}")

if __name__ == "__main__":
    #deals.json url
    deals = load_json("https://raw.githubusercontent.com/SpiffInc/spiff_data_engineering_exercises/main/data/deals.json")
    #products.json url
    products = load_json("https://raw.githubusercontent.com/SpiffInc/spiff_data_engineering_exercises/main/data/products.json")

    #convert the deals.json file to a pandas dataframe
    deals_df = JsonToDfConverter(deals).convert()

    #convert the products.json file to a pandas dataframe
    products_df = JsonToDfConverter(products).convert()

    # Merge the deals_df and products_df dataframes on the product_id column
    merged_df = DFJoiner(deals_df, products_df, 'product_id').join()

    #write the merged results to csv
    merged_df.to_csv('sales_data.csv', index=False)

    #calculate and print the total commission for sales_rep_name 'Ian' between 2023-01-01 and 2023-4-30
    PrintCommission('Ian','2023-01-01', '2023-4-30')
    
    #calculate  and print the total commission for sales_rep_name 'David' between 2023-04-01 and 2023-06-30
    PrintCommission('David','2023-04-01', '2023-06-30')

    #calculate and print the total commission for sales_rep_name 'Poppy' between 2023-03-01 and 2023-5-30
    PrintCommission('Poppy','2023-03-01', '2023-5-30')

Commission for Ian between 2023-01-01 and 2023-4-30 is 55350.00
Commission for David between 2023-04-01 and 2023-06-30 is 89540.00
Commission for Poppy between 2023-03-01 and 2023-5-30 is 118190.00


In [52]:
import unittest

class TestCommissionStrategy(unittest.TestCase):
    
    def test_fixed_commission_strategy(self):
        data = QueryData("Ian", "2023-01-01", "2023-4-30")
        commissionStrategyFactory = CommissionStrategyFactory()
        calcuateCommission = CalcuateCommission(commissionStrategyFactory)
        commission = calcuateCommission.CalculateCommission(data,"Fixed")
        self.assertEqual(commission, '55350.00')

    def test_invalid_commission_strategy(self):
        data = QueryData("Ian", "2023-01-01", "2023-4-30")
        commissionStrategyFactory = CommissionStrategyFactory()
        calcuateCommission = CalcuateCommission(commissionStrategyFactory)
        with self.assertRaises(Exception):
            calcuateCommission.CalculateCommission(data,"Invalid")
    
    def test_json_to_df_converter(self):
        deals = load_json("https://raw.githubusercontent.com/SpiffInc/spiff_data_engineering_exercises/main/data/deals.json")
        deals_df = JsonToDfConverter(deals).convert()
        self.assertEqual(deals_df.shape, (50, 6))

    def test_df_joiner(self):
        deals = load_json("https://raw.githubusercontent.com/SpiffInc/spiff_data_engineering_exercises/main/data/deals.json")
        products = load_json("https://raw.githubusercontent.com/SpiffInc/spiff_data_engineering_exercises/main/data/products.json")
        deals_df = JsonToDfConverter(deals).convert()
        products_df = JsonToDfConverter(products).convert()
        merged_df = DFJoiner(deals_df, products_df, 'product_id').join()
        self.assertEqual(merged_df.shape, (50, 9))
    
    def test_load_json(self):
        deals = load_json("https://raw.githubusercontent.com/SpiffInc/spiff_data_engineering_exercises/main/data/deals.json")
        self.assertEqual(len(deals), 50)
    
    def test_calculate_commission(self):
        commission = calculate_commission('Ian', '2023-01-01', '2023-4-30')
        self.assertEqual(commission, '55350.00')

    def test_validate_data(self):
        data = QueryData("Ian", "2023-01-01", "2023-4-30")
        self.assertEqual(validate_data(data), True)

    def test_empty_sales_rep_name(self):
        data = QueryData("", "2023-01-01", "2023-4-30")
        with self.assertRaises(Exception) as e:
            validate_data(data)
        self.assertEqual(str(e.exception), "data cannot be empty")
    
    def test_empty_start_date(self):
        data = QueryData("Ian", "", "2023-4-30")
        with self.assertRaises(Exception) as e:
            validate_data(data)
        self.assertEqual(str(e.exception), "data cannot be empty")
    
    def test_empty_end_date(self):
        data = QueryData("Ian", "2023-01-01", "")
        with self.assertRaises(Exception) as e:
            validate_data(data)
        self.assertEqual(str(e.exception), "data cannot be empty")
    
    def test_start_date_greater_than_end_date(self):
        data = QueryData("Ian", "2023-04-30", "2023-01-01")
        with self.assertRaises(Exception) as e:
            validate_data(data)
        self.assertEqual(str(e.exception), "start_date must be before end_date")

    def test_start_date_format(self):
        data = QueryData("Ian", "01-2022-01", "2023-4-30")
        with self.assertRaises(Exception) as e:
            validate_data(data)
        self.assertEqual(str(e.exception), "start_date must be in the format YYYY-MM-DD")

    def test_end_date_format(self):
        data = QueryData("Ian", "2023-01-01", "4-2021-3")
        with self.assertRaises(Exception) as e:
            validate_data(data)
        self.assertEqual(str(e.exception), "end_date must be in the format YYYY-MM-DD")

    def test_sales_rep_name_not_exist(self):
        data = QueryData("Ian1", "2023-01-01", "2023-4-30")
        with self.assertRaises(Exception) as e:
            validate_data(data)
        self.assertEqual(str(e.exception), "sales_rep_name must be in the sales_data.csv file")

unittest.main(argv=[''], verbosity=2, exit=False)

test_calculate_commission (__main__.TestCommissionStrategy) ... ok
test_df_joiner (__main__.TestCommissionStrategy) ... ok
test_empty_end_date (__main__.TestCommissionStrategy) ... ok
test_empty_sales_rep_name (__main__.TestCommissionStrategy) ... ok
test_empty_start_date (__main__.TestCommissionStrategy) ... ok
test_end_date_format (__main__.TestCommissionStrategy) ... ok
test_fixed_commission_strategy (__main__.TestCommissionStrategy) ... ok
test_invalid_commission_strategy (__main__.TestCommissionStrategy) ... ok
test_json_to_df_converter (__main__.TestCommissionStrategy) ... ok
test_load_json (__main__.TestCommissionStrategy) ... ok
test_sales_rep_name_not_exist (__main__.TestCommissionStrategy) ... ok
test_start_date_format (__main__.TestCommissionStrategy) ... ok
test_start_date_greater_than_end_date (__main__.TestCommissionStrategy) ... ok
test_validate_data (__main__.TestCommissionStrategy) ... ok

----------------------------------------------------------------------
Ran 14 te